# Hyperopt

In [23]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from scipy.stats import loguniform
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import hyperopt.pyll.stochastic
import seaborn as sns
import matplotlib as plt
from sklearn.metrics import accuracy_score

### Load and Format Data

In [2]:
os.chdir('/Users/johnoliver/Downloads/grad-nba-wins/data/mod_data')
# data from 2010
df10 = pd.read_csv("mod10.csv")
# get rid of first variable (unique identifier)
df10 = df10.iloc[:,1:]

In [3]:
# define x and y variables
feature_cols = ["h_avg_points","a_avg_points","h_avg_fg","a_avg_fg",
                "h_avg_fga","a_avg_fga","h_avg_3p","a_avg_3p",
                "h_avg_3pa","a_avg_3pa","h_avg_ft","a_avg_ft",
                "h_avg_orb","a_avg_orb","h_avg_drb" ,"a_avg_drb",
                "h_avg_ast","a_avg_ast","h_avg_stl", "a_avg_stl", 
                "h_avg_blk","a_avg_blk","h_avg_tov","a_avg_tov",
                "h_avg_pf","a_avg_pf" ,"h_avg_tsp",  "a_avg_tsp",
                "h_avg_ortg","a_avg_ortg","h_avg_drtg","a_avg_drtg",
                "h_win_perc","a_win_perc","h_back","a_back",
                "home_elo", "away_elo"]

X = df10[feature_cols]
Y = df10.win_status

scaler = StandardScaler()
scale_X = scaler.fit_transform(X)

# split data into training and testing
X_train,X_test,y_train,y_test=train_test_split(X,Y,
                                               test_size=0.3,
                                               random_state=0)
# scaled data
scale_X_train,scale_X_test,y_train,y_test=train_test_split(scale_X,Y,
                                               test_size=0.3,
                                               random_state=0)

# Logistic Regression

In [38]:
%%time
def hyperopt_train_test(params):
    X_ = X_train[:]
    clf = LogisticRegression(**params)
    return cross_val_score(clf, X_, y_train).mean()

space4lr = {
    'solver': hp.choice('solver', ['newton-cg', 'lbfgs', 'liblinear']),
    'penalty': hp.choice('penalty', ['none', 'l1', 'l2', 'elasticnet']),
    'C': hp.uniform('C', 0, 20),
    'multi_class': hp.choice('multi_class', ["auto", "ovr"]),
    'verbose': hp.choice('verbose', [0,1,2])
}

best = 0
def f(params):
    global best
    acc = hyperopt_train_test(params)
    if acc > best:
        best = acc
    print('new best:', best, params)
    return {'loss': -acc, 'status': STATUS_OK}

trials = Trials()
best = fmin(f, space4lr, algo=tpe.suggest, max_evals=300, trials=trials)
print('best:')
print(best)


new best:                                                                       
0                                                                               
{'C': 6.200233095470711, 'multi_class': 'auto', 'penalty': 'l1', 'solver': 'newton-cg', 'verbose': 0}
[LibLinear]                                                                     
  0%|                           | 1/300 [00:00<00:38,  7.73trial/s, best loss=?]iter  1 act 1.315e+02 pre 1.304e+02 delta 2.094e-04 f 3.664e+03 |g| 1.256e+06 CG   1
cg reaches trust region boundary
iter  2 act 2.317e+01 pre 2.317e+01 delta 8.375e-04 f 3.533e+03 |g| 1.168e+05 CG   1
cg reaches trust region boundary
iter  3 act 8.700e+01 pre 8.712e+01 delta 3.350e-03 f 3.510e+03 |g| 1.445e+05 CG   2
cg reaches trust region boundary
iter  4 act 2.269e+02 pre 2.220e+02 delta 5.428e-03 f 3.423e+03 |g| 1.579e+05 CG   2
iter  5 act 5.209e+01 pre 4.905e+01 delta 5.428e-03 f 3.196e+03 |g| 4.494e+04 CG   2
cg reaches trust region boundary
iter  6 act 1.115e

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/Library/Frameworks/Python.fra

cg reaches trust region boundary
iter  9 act 5.401e+01 pre 5.329e+01 delta 9.002e-02 f 3.072e+03 |g| 1.684e+03 CG   3
iter 10 act 2.941e-01 pre 2.920e-01 delta 9.002e-02 f 3.018e+03 |g| 1.234e+04 CG   2
cg reaches trust region boundary
iter 11 act 3.383e+01 pre 3.386e+01 delta 1.011e-01 f 3.018e+03 |g| 7.614e+02 CG   5
iter 12 act 5.587e-02 pre 5.573e-02 delta 1.011e-01 f 2.984e+03 |g| 6.007e+03 CG   2
cg reaches trust region boundary
iter 13 act 2.108e+01 pre 2.067e+01 delta 1.060e-01 f 2.984e+03 |g| 5.928e+02 CG   7
iter 14 act 7.003e+00 pre 7.050e+00 delta 1.060e-01 f 2.963e+03 |g| 3.141e+03 CG   3
iter 15 act 2.703e+00 pre 2.715e+00 delta 1.060e-01 f 2.956e+03 |g| 3.608e+03 CG   4
cg reaches trust region boundary
iter 16 act 1.216e+01 pre 1.196e+01 delta 1.171e-01 f 2.953e+03 |g| 8.898e+02 CG   6
iter 17 act 1.214e-01 pre 1.215e-01 delta 1.171e-01 f 2.941e+03 |g| 3.338e+04 CG   1
cg reaches trust region boundary
iter 18 act 1.226e+01 pre 1.228e+01 delta 1.533e-01 f 2.941e+03 |g| 4.

cg reaches trust region boundary
iter 27 act 1.682e+00 pre 1.685e+00 delta 2.630e-01 f 2.864e+03 |g| 1.038e+02 CG  27
iter 28 act 3.086e-05 pre 3.086e-05 delta 2.630e-01 f 2.862e+03 |g| 5.251e+02 CG   1
[LibLinear]                                                                     
  0%|                           | 1/300 [00:00<02:08,  2.32trial/s, best loss=?]iter  1 act 6.071e+02 pre 5.636e+02 delta 7.302e-03 f 3.671e+03 |g| 1.257e+06 CG   2
iter  2 act 1.918e+01 pre 1.818e+01 delta 7.302e-03 f 3.064e+03 |g| 1.710e+05 CG   2
cg reaches trust region boundary
iter  3 act 1.272e+01 pre 1.270e+01 delta 2.565e-02 f 3.045e+03 |g| 1.382e+04 CG   3
cg reaches trust region boundary
iter  4 act 3.691e+01 pre 3.695e+01 delta 8.222e-02 f 3.032e+03 |g| 2.734e+03 CG   3
cg reaches trust region boundary
iter  5 act 5.291e+01 pre 5.127e+01 delta 8.690e-02 f 2.995e+03 |g| 3.070e+03 CG   4
iter  6 act 2.070e-01 pre 2.056e-01 delta 8.690e-02 f 2.942e+03 |g| 1.725e+04 CG   2
cg reaches trust region bou

iter  43  #CD cycles 1000
iter  44  #CD cycles 1000
iter  45  #CD cycles 1000
iter  46  #CD cycles 1000
iter  47  #CD cycles 1000
iter  48  #CD cycles 1000
iter  49  #CD cycles 1000
iter  50  #CD cycles 471
iter  51  #CD cycles 89
iter  52  #CD cycles 779
iter  53  #CD cycles 876
iter  54  #CD cycles 72
iter  55  #CD cycles 112
iter  56  #CD cycles 304
iter  57  #CD cycles 1000
iter  58  #CD cycles 21
iter  59  #CD cycles 176
iter  60  #CD cycles 25
iter  61  #CD cycles 273
iter  62  #CD cycles 3
iter  63  #CD cycles 41
iter  64  #CD cycles 18
iter  65  #CD cycles 35
iter  66  #CD cycles 19
iter  67  #CD cycles 6
iter  68  #CD cycles 30
iter  69  #CD cycles 1
iter  70  #CD cycles 1000
iter  71  #CD cycles 1000
iter  72  #CD cycles 1000
iter  73  #CD cycles 1000
iter  74  #CD cycles 1000
iter  75  #CD cycles 1000
iter  76  #CD cycles 1000
optimization finished, #iter = 76
Objective value = 2250.896586
#nonzeros/#features = 36/39
[LibLinear]                                               

...

new best:                                                                       
0.7148509763617676                                                              
{'C': 19.43062375898637, 'multi_class': 'ovr', 'penalty': 'elasticnet', 'solver': 'lbfgs', 'verbose': 2}
 99%|█████▉| 297/300 [00:50<00:00, 12.85trial/s, best loss: -0.7148509763617676]

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/Library/Frameworks/Python.fra

new best:                                                                       
0.7148509763617676                                                              
{'C': 19.750929963527085, 'multi_class': 'ovr', 'penalty': 'elasticnet', 'solver': 'lbfgs', 'verbose': 2}
new best:                                                                       


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/Library/Frameworks/Python.fra

0.7148509763617676
{'C': 19.970231885243365, 'multi_class': 'ovr', 'penalty': 'elasticnet', 'solver': 'lbfgs', 'verbose': 2}
100%|██████| 300/300 [00:50<00:00,  5.88trial/s, best loss: -0.7148509763617676]
best:
{'C': 19.25119239476978, 'multi_class': 1, 'penalty': 2, 'solver': 1, 'verbose': 2}
CPU times: user 53.2 s, sys: 4.96 s, total: 58.2 s
Wall time: 51 s


In [39]:
# define model
lr_model = LogisticRegression(C = 19.25119239476978, multi_class = 'auto', penalty = 'l2',
                              solver = 'lbfgs', verbose = 2)
lr_model.fit(X_train, y_train)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           39     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.83817D+02    |proj g|=  1.28142D+05

At iterate    1    f=  4.68211D+02    |proj g|=  5.14627D+04

At iterate    2    f=  4.65828D+02    |proj g|=  1.45660D+04

At iterate    3    f=  4.65382D+02    |proj g|=  1.43057D+04

At iterate    4    f=  4.61726D+02    |proj g|=  4.17664D+04

At iterate    5    f=  4.55356D+02    |proj g|=  6.87764D+04

At iterate    6    f=  4.41141D+02    |proj g|=  9.63668D+04

At iterate    7    f=  4.22634D+02    |proj g|=  9.12924D+04

At iterate    8    f=  4.05170D+02    |proj g|=  1.34848D+04

At iterate    9    f=  4.04415D+02    |proj g|=  1.29916D+03

At iterate   10    f=  4.04376D+02    |proj g|=  4.20769D+02

At iterate   11    f=  4.04375D+02    |proj g|=  1.77297D+02

At iterate   12    f=  4.04373D+02    |proj g|=  1.52457D+02

At iterate   13    f=  4.0

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
 This problem is unconstrained.
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


LogisticRegression(C=19.25119239476978, verbose=2)


At iterate  100    f=  3.83989D+02    |proj g|=  2.28584D+02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   39    100    117      1     0     0   2.286D+02   3.840D+02
  F =   383.98852912876237     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 


In [40]:
lr_ypred = lr_model.predict(X_test)
print('Test Score: %s' % accuracy_score(y_test, lr_ypred))

Test Score: 0.63


# Random Forest

In [ ]:
%%time
def hyperopt_train_test(params):
    X_ = X_train[:]
    clf = RandomForestClassifier(**params)
    return cross_val_score(clf, X_, y_train).mean()

space4rf = {
    'max_depth': hp.choice('max_depth', range(1,20)),
    'max_features': hp.choice('max_features', range(1,5)),
    'n_estimators': hp.choice('n_estimators', range(1,20)),
    'criterion': hp.choice('criterion', ["gini", "entropy"])
}

best = 0
def f(params):
    global best
    acc = hyperopt_train_test(params)
    if acc > best:
        best = acc
    print('new best:', best, params)
    return {'loss': -acc, 'status': STATUS_OK}

trials = Trials()
best = fmin(f, space4rf, algo=tpe.suggest, max_evals=300, trials=trials)
print('best:')
print(best)

In [30]:
#define model
rf_model = RandomForestClassifier(max_depth = 3, max_features = 3, n_estimators = 16)
rf_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=3, max_features=3, n_estimators=16)

In [32]:
rf_ypred = rf_model.predict(X_test)
print('Test Score: %s' % accuracy_score(y_test, rf_ypred))

Test Score: 0.6766666666666666


# Support Vector Machine

In [12]:
%%time
def hyperopt_train_test(params):
    X_ = scale_X_train[:]
    clf = SVC(**params)
    return cross_val_score(clf, X_, y_train).mean()

space4svm = {
    'C': hp.uniform('C', 0, 20),
    'kernel': hp.choice('kernel', ['linear', 'sigmoid', 'poly', 'rbf']),
    'gamma': hp.uniform('gamma', 0, 20)
}

def f(params):
    acc = hyperopt_train_test(params)
    return {'loss': -acc, 'status': STATUS_OK}

trials = Trials()
best = fmin(f, space4svm, algo=tpe.suggest, max_evals=100, trials=trials)
print('best:')
print(best)

100%|███████| 100/100 [01:44<00:00,  1.05s/trial, best loss: -0.706289825282631]
best:
{'C': 0.8062088239045105, 'gamma': 19.733920589167205, 'kernel': 0}
CPU times: user 1min 32s, sys: 951 ms, total: 1min 33s
Wall time: 1min 44s


In [20]:
svc_model = SVC(C = 0.8062088239045105,
                gamma = 19.733920589167205,
                kernel = 'linear')

svc_model.fit(scale_X_train,y_train)

SVC(C=0.8062088239045105, gamma=19.733920589167205, kernel='linear')

In [24]:
svc_ypred = svc_model.predict(X_test)
print('Test Score: %s' % accuracy_score(y_test, svc_ypred))

Test Score: 0.5066666666666667


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(
